# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa7e66c1ca0>
├── 'a' --> tensor([[-2.0730, -1.0968,  1.9922],
│                   [-0.6148, -1.3081, -1.6231]])
└── 'x' --> <FastTreeValue 0x7fa7e462a2b0>
    └── 'c' --> tensor([[-0.2482,  0.6803, -1.2051,  0.6468],
                        [ 0.3537,  0.5891, -0.3867, -0.4435],
                        [-1.0667,  0.2462,  0.4483,  0.3852]])

In [4]:
t.a

tensor([[-2.0730, -1.0968,  1.9922],
        [-0.6148, -1.3081, -1.6231]])

In [5]:
%timeit t.a

66 ns ± 0.041 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa7e66c1ca0>
├── 'a' --> tensor([[-1.5753,  0.3139, -0.5082],
│                   [-0.8841,  1.7201,  0.2743]])
└── 'x' --> <FastTreeValue 0x7fa7e462a2b0>
    └── 'c' --> tensor([[-0.2482,  0.6803, -1.2051,  0.6468],
                        [ 0.3537,  0.5891, -0.3867, -0.4435],
                        [-1.0667,  0.2462,  0.4483,  0.3852]])

In [7]:
%timeit t.a = new_value

68.1 ns ± 0.029 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.0730, -1.0968,  1.9922],
               [-0.6148, -1.3081, -1.6231]]),
    x: Batch(
           c: tensor([[-0.2482,  0.6803, -1.2051,  0.6468],
                      [ 0.3537,  0.5891, -0.3867, -0.4435],
                      [-1.0667,  0.2462,  0.4483,  0.3852]]),
       ),
)

In [10]:
b.a

tensor([[-2.0730, -1.0968,  1.9922],
        [-0.6148, -1.3081, -1.6231]])

In [11]:
%timeit b.a

59.6 ns ± 0.0349 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5066,  0.2011,  0.0824],
               [-1.3101,  0.3871, -0.2955]]),
    x: Batch(
           c: tensor([[-0.2482,  0.6803, -1.2051,  0.6468],
                      [ 0.3537,  0.5891, -0.3867, -0.4435],
                      [-1.0667,  0.2462,  0.4483,  0.3852]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.256 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

843 ns ± 0.135 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 7.79 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 605 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 370 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa70c3fa8b0>
├── 'a' --> tensor([[[-2.0730, -1.0968,  1.9922],
│                    [-0.6148, -1.3081, -1.6231]],
│           
│                   [[-2.0730, -1.0968,  1.9922],
│                    [-0.6148, -1.3081, -1.6231]],
│           
│                   [[-2.0730, -1.0968,  1.9922],
│                    [-0.6148, -1.3081, -1.6231]],
│           
│                   [[-2.0730, -1.0968,  1.9922],
│                    [-0.6148, -1.3081, -1.6231]],
│           
│                   [[-2.0730, -1.0968,  1.9922],
│                    [-0.6148, -1.3081, -1.6231]],
│           
│                   [[-2.0730, -1.0968,  1.9922],
│                    [-0.6148, -1.3081, -1.6231]],
│           
│                   [[-2.0730, -1.0968,  1.9922],
│                    [-0.6148, -1.3081, -1.6231]],
│           
│                   [[-2.0730, -1.0968,  1.9922],
│                    [-0.6148, -1.3081, -1.6231]]])
└── 'x' --> <FastTreeValue 0x7fa70c3fa7c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 45.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa714274c70>
├── 'a' --> tensor([[-2.0730, -1.0968,  1.9922],
│                   [-0.6148, -1.3081, -1.6231],
│                   [-2.0730, -1.0968,  1.9922],
│                   [-0.6148, -1.3081, -1.6231],
│                   [-2.0730, -1.0968,  1.9922],
│                   [-0.6148, -1.3081, -1.6231],
│                   [-2.0730, -1.0968,  1.9922],
│                   [-0.6148, -1.3081, -1.6231],
│                   [-2.0730, -1.0968,  1.9922],
│                   [-0.6148, -1.3081, -1.6231],
│                   [-2.0730, -1.0968,  1.9922],
│                   [-0.6148, -1.3081, -1.6231],
│                   [-2.0730, -1.0968,  1.9922],
│                   [-0.6148, -1.3081, -1.6231],
│                   [-2.0730, -1.0968,  1.9922],
│                   [-0.6148, -1.3081, -1.6231]])
└── 'x' --> <FastTreeValue 0x7fa7142dcfa0>
    └── 'c' --> tensor([[-0.2482,  0.6803, -1.2051,  0.6468],
                        [ 0.3537,  0.5891, -0.3867, -0.4435],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 38.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.6 µs ± 49.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.2482,  0.6803, -1.2051,  0.6468],
                       [ 0.3537,  0.5891, -0.3867, -0.4435],
                       [-1.0667,  0.2462,  0.4483,  0.3852]],
              
                      [[-0.2482,  0.6803, -1.2051,  0.6468],
                       [ 0.3537,  0.5891, -0.3867, -0.4435],
                       [-1.0667,  0.2462,  0.4483,  0.3852]],
              
                      [[-0.2482,  0.6803, -1.2051,  0.6468],
                       [ 0.3537,  0.5891, -0.3867, -0.4435],
                       [-1.0667,  0.2462,  0.4483,  0.3852]],
              
                      [[-0.2482,  0.6803, -1.2051,  0.6468],
                       [ 0.3537,  0.5891, -0.3867, -0.4435],
                       [-1.0667,  0.2462,  0.4483,  0.3852]],
              
                      [[-0.2482,  0.6803, -1.2051,  0.6468],
                       [ 0.3537,  0.5891, -0.3867, -0.4435],
                       [-1.0667,  0.2462,  0.4483,  0.3852]],

In [26]:
%timeit Batch.stack(batches)

79.5 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.2482,  0.6803, -1.2051,  0.6468],
                      [ 0.3537,  0.5891, -0.3867, -0.4435],
                      [-1.0667,  0.2462,  0.4483,  0.3852],
                      [-0.2482,  0.6803, -1.2051,  0.6468],
                      [ 0.3537,  0.5891, -0.3867, -0.4435],
                      [-1.0667,  0.2462,  0.4483,  0.3852],
                      [-0.2482,  0.6803, -1.2051,  0.6468],
                      [ 0.3537,  0.5891, -0.3867, -0.4435],
                      [-1.0667,  0.2462,  0.4483,  0.3852],
                      [-0.2482,  0.6803, -1.2051,  0.6468],
                      [ 0.3537,  0.5891, -0.3867, -0.4435],
                      [-1.0667,  0.2462,  0.4483,  0.3852],
                      [-0.2482,  0.6803, -1.2051,  0.6468],
                      [ 0.3537,  0.5891, -0.3867, -0.4435],
                      [-1.0667,  0.2462,  0.4483,  0.3852],
                      [-0.2482,  0.6803, -1.2051,  0.6468],
                   

In [28]:
%timeit Batch.cat(batches)

148 µs ± 513 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

320 µs ± 4.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
